In [2]:
import duckdb

In [3]:
con = duckdb.connect()
for ext in ["postgres", "spatial"]:
    con.install_extension(ext)
    con.load_extension(ext)
con.sql("""
    ATTACH 'dbname=spartid_ais user=postgres password=postgres host=127.0.0.1' AS db (TYPE postgres);
""")

In [ ]:
con.sql("SELECT COUNT(*) FROM db.public.last_position")

In [ ]:
con.sql("SELECT COUNT(*) FROM db.public.historic_position")

In [27]:
con.sql("COPY db.public.last_position to 'last_position.parquet' (FORMAT PARQUET);")

In [28]:
con.sql(
    "COPY db.public.imo_vessel_codes to 'imo_vessel_codes.parquet' (FORMAT PARQUET);"
)

In [6]:
con.sql(
    "COPY db.public.historic_position to 'historic_position.parquet' (FORMAT PARQUET);"
)

In [20]:
con.sql("""
    CREATE OR REPLACE VIEW historic_groups AS
    SELECT
        EXTRACT(YEAR FROM timestamp) as year,
        EXTRACT(MONTH FROM timestamp) as month,
        EXTRACT(WEEK FROM timestamp) as week,
        timestamp,
        *,
    FROM
        db.public.historic_position
        ;
""")

In [26]:
year = 2024
for week in range(10, 52):
    con.sql(f"""
        COPY (
            SELECT *
            FROM historic_groups
            WHERE year = '{year}' AND week = '{week}'
        ) TO historic_position_{year}_w{week}.parquet (FORMAT PARQUET)
        ;
    """)

In [ ]:
a = con.sql("SELECT * from historic_groups;")
a